# Week 5 - Final Assignment

## Description of the problem and a discussion of the background.

A friend of mine is a good cook and moving from another city to Chicago with a plan to open a restaurant. He asked my help to find a best community that fits his needs. His requirements are, The place and surroundings should have sizable middle age people and good employment rate. Also since he is going to open a Mexican restaurant there should be a sizable Latino and Hispanic community in the neighborhood but not with many Mexican restaurants around. Also his restaurant price range is going to be Moderate to High so the community income should be decent to afford.

## Datasets

To Solve the problem, I need the following datasets:
    
    1) As the restuarant is going to carry moderate to high price range dishes , I need the Communities in Chicago, their employment rates and per capita income

    2) As the need is to open in a community with decent employment rates and sizeable Latino community and middle age groups, I need a dataset with demographic information with age.

    3) I need to query the foursquare API to address the final need, make sure there are not many Mexican resturants arounds and their price ranges to get an idea of the community


After careful research, Here are a few datasets I have found that helps me find the best place that fits his needs:
    
    Census data to help with first dataset need above:
    
        https://datahub.cmap.illinois.gov/dataset/2010-census-data-summarized-to-chicago-community-areas
    
    Census data to help with first dataset need above:
    
        https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2

    Finally Foursquare API for price ranges and to find other Mexican restuarants with similar price ranges.

### Plan of execution:
Step 1: Import the two datasets as CSV, remove the attributes that are not needed, merge some attributes as per need and changes cases/types

Step 2: Create three Ranking columns 
    
        1) Rank on the Hispanic and latino population counts 
        
        2) Rank on Income of the community 
        
        3) Rank on the age groups

Step 3: Connect to foursquare and create a list of all the mexican restuarants and their price tier by community

Step 3: Combine the foursquare data and the community data and find the top two communities that really fit the requirements


### Step 1: 
### Import the two datasets as CSV, remove the attributes that are not needed, merge some attributes as per need and changes cases/types


In [1]:

import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

from sklearn.cluster import KMeans 

print('Libraries imported.')


Libraries imported.


In [2]:
### In this step we are importing the CSV which contains the demographic information about each community
Chicago_Neigh_Demo = pd.read_csv('Demographics_Chicago_2008_2012.csv')
Chicago_Neigh_Demo

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,14.0,Albany Park,11.3,19.2,10.0,32.9,32.0,21323,53.0
1,57.0,Archer Heights,8.5,14.1,16.5,35.9,39.2,16134,67.0
2,34.0,Armour Square,5.7,40.1,16.7,34.5,38.3,16148,82.0
3,70.0,Ashburn,4.0,10.4,11.7,17.7,36.9,23482,37.0
4,71.0,Auburn Gresham,4.0,27.6,28.3,18.5,41.9,15528,74.0
...,...,...,...,...,...,...,...,...,...
73,65.0,West Lawn,5.8,14.9,9.6,33.6,39.6,16907,56.0
74,53.0,West Pullman,3.3,25.9,19.4,20.5,42.1,16563,62.0
75,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
76,24.0,West Town,2.3,14.7,6.6,12.9,21.7,43198,10.0


In [3]:
!pip install xlrd
### In this step we are importing an excel which contains the Community information and Hispanic and Latino community details in particular
Chicago_Neigh_Data = pd.read_excel (r'Chicago_Neighborhoods.xlsx') 
Chicago_Neigh = pd.DataFrame(Chicago_Neigh_Data, columns = ['Geog','GeogKey','Total Population','Not Hispanic or Latino but White alone','Not Hispanic or Latino but Black or African American alone','Not Hispanic or Latino but American Indian and Alaska Native alone','Not Hispanic or Latino but Asian alone','Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone','Not Hispanic or Latino but Some Other Race alone','Not Hispanic or Latino but Two or More Races','Hispanic or Latino','Male:  to 5 yrs','Male: 5 to 9 yrs','Male: 10 to 14 yrs','Male: 15 to 17 yrs','Male: 18 and 19 yrs','Male: 20 yrs','Male: 21 yrs','Male: 22 to 24 yrs','Male: 25 to 29 yrs','Male: 30 to 34 yrs','Male: 35 to 39 yrs','Male: 40 to 44 yrs','Male: 45 to 49 yrs','Male: 50 to 54 yrs','Male: 55 to 59 yrs','Male: 60 to 61 yrs','Male: 62 to 64 yrs','Male: 65 to 66 yrs','Male: 67 to 69 yrs','Male: 70 to 74 yrs','Male: 75 to 79 yrs','Male: 80 to 84 yrs','Male: 85 yrs and over','FeMale: Under 5 yrs','FeMale: 5 to 9 yrs','FeMale: 10 to 14 yrs','FeMale: 15 to 17 yrs','FeMale: 18 to 19 yrs','FeMale: 20 yrs','FeMale: 21 yrs','FeMale: 22 to 24 yrs','FeMale: 25 to 29 yrs','FeMale: 30 to 34 yrs','FeMale: 35 to 39 yrs','FeMale: 40 to 44 yrs','FeMale: 45 to 49 yrs','FeMale: 50 to 54 yrs','FeMale: 55 to 59 yrs','FeMale: 60 to 61 yrs','FeMale: 62 to 64 yrs','FeMale: 65 to 66 yrs','FeMale: 67 to 69 yrs','FeMale: 70 to 74 yrs','FeMale: 75 to 79 yrs','FeMale: 80 to 84 yrs','FeMale: 85 yrs and over','Median Age','Total Households'])
Chicago_Neigh.head(5)

     |████████████████████████████████| 112kB 2.8MB/s eta 0:00:01


,Geog,GeogKey,Total Population,Not Hispanic or Latino but White alone,Not Hispanic or Latino but Black or African American alone,Not Hispanic or Latino but American Indian and Alaska Native alone,Not Hispanic or Latino but Asian alone,Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone,Not Hispanic or Latino but Some Other Race alone,Not Hispanic or Latino but Two or More Races,...,FeMale: 60 to 61 yrs,FeMale: 62 to 64 yrs,FeMale: 65 to 66 yrs,FeMale: 67 to 69 yrs,FeMale: 70 to 74 yrs,FeMale: 75 to 79 yrs,FeMale: 80 to 84 yrs,FeMale: 85 yrs and over,Median Age,Total Households
0,Rogers Park,1,54991,21618,14461,142,3523,39,202,1573,...,486,661,308,383,534,392,312,388,32.404941,24292
1,West Ridge,2,71942,30706,8015,189,16184,3,225,1919,...,794,1026,561,718,1056,799,713,931,35.062344,24755
2,Uptown,3,56362,29098,11275,191,6414,10,130,1235,...,561,679,360,475,763,620,524,690,35.464521,29719
3,Lincoln Square,4,39493,24916,1488,72,4399,11,128,917,...,322,493,256,323,436,340,275,316,33.450642,18194
4,North Center,5,31867,24634,738,55,1449,16,51,594,...,261,314,159,218,319,257,181,203,32.478928,14561


In [4]:
# Cleaning up of the datasets - Demographics data
##Cleaning up includes, removing those age groups that eat in a restuarant very less, ages < 15yrs and < 80yrs
##Male Cleaning up includes, merging some columns and dropping after merge 
Chicago_Neigh.drop(['Male:  to 5 yrs','Male: 5 to 9 yrs','Male: 10 to 14 yrs'],axis = 1, inplace = True)
Chicago_Neigh['Male: 15 to 70 yrs'] = Chicago_Neigh['Male: 15 to 17 yrs'] + Chicago_Neigh['Male: 18 and 19 yrs'] + Chicago_Neigh['Male: 20 yrs'] + Chicago_Neigh['Male: 21 yrs'] + Chicago_Neigh['Male: 22 to 24 yrs'] + Chicago_Neigh['Male: 25 to 29 yrs'] + Chicago_Neigh['Male: 30 to 34 yrs'] + Chicago_Neigh['Male: 35 to 39 yrs'] + Chicago_Neigh['Male: 40 to 44 yrs'] + Chicago_Neigh['Male: 45 to 49 yrs'] + Chicago_Neigh['Male: 50 to 54 yrs'] + Chicago_Neigh['Male: 55 to 59 yrs'] + Chicago_Neigh['Male: 60 to 61 yrs']+ Chicago_Neigh['Male: 62 to 64 yrs'] + Chicago_Neigh['Male: 65 to 66 yrs'] + Chicago_Neigh['Male: 67 to 69 yrs']
Chicago_Neigh.drop(['Male: 15 to 17 yrs','Male: 18 and 19 yrs','Male: 20 yrs','Male: 21 yrs','Male: 22 to 24 yrs','Male: 25 to 29 yrs','Male: 30 to 34 yrs','Male: 35 to 39 yrs','Male: 40 to 44 yrs','Male: 45 to 49 yrs','Male: 50 to 54 yrs','Male: 55 to 59 yrs','Male: 70 to 74 yrs','Male: 75 to 79 yrs','Male: 80 to 84 yrs','Male: 60 to 61 yrs','Male: 62 to 64 yrs','Male: 65 to 66 yrs','Male: 67 to 69 yrs','Male: 85 yrs and over'],axis = 1, inplace = True)
##Female Cleaning up includes, merging some columns and dropping after merge 
Chicago_Neigh.drop(['FeMale: Under 5 yrs','FeMale: 5 to 9 yrs','FeMale: 10 to 14 yrs','FeMale: 70 to 74 yrs','FeMale: 75 to 79 yrs','FeMale: 80 to 84 yrs','FeMale: 85 yrs and over'],axis = 1, inplace = True)
Chicago_Neigh['Female: 15 to 70 yrs'] = Chicago_Neigh['FeMale: 15 to 17 yrs']+Chicago_Neigh['FeMale: 18 to 19 yrs']+Chicago_Neigh['FeMale: 20 yrs']+Chicago_Neigh['FeMale: 21 yrs']+Chicago_Neigh['FeMale: 22 to 24 yrs']+Chicago_Neigh['FeMale: 25 to 29 yrs']+Chicago_Neigh['FeMale: 30 to 34 yrs']+Chicago_Neigh['FeMale: 35 to 39 yrs']+Chicago_Neigh['FeMale: 40 to 44 yrs']+Chicago_Neigh['FeMale: 45 to 49 yrs']+Chicago_Neigh['FeMale: 50 to 54 yrs']+Chicago_Neigh['FeMale: 55 to 59 yrs']+Chicago_Neigh['FeMale: 60 to 61 yrs']+Chicago_Neigh['FeMale: 62 to 64 yrs']+Chicago_Neigh['FeMale: 65 to 66 yrs']+Chicago_Neigh['FeMale: 67 to 69 yrs']
Chicago_Neigh.drop(['FeMale: 15 to 17 yrs','FeMale: 18 to 19 yrs','FeMale: 20 yrs','FeMale: 21 yrs','FeMale: 22 to 24 yrs','FeMale: 25 to 29 yrs','FeMale: 30 to 34 yrs','FeMale: 35 to 39 yrs','FeMale: 40 to 44 yrs','FeMale: 45 to 49 yrs','FeMale: 50 to 54 yrs','FeMale: 55 to 59 yrs','FeMale: 60 to 61 yrs','FeMale: 62 to 64 yrs','FeMale: 65 to 66 yrs','FeMale: 67 to 69 yrs'],axis = 1, inplace = True)
## Merging all communities
Chicago_Neigh['Not Hispanic or Latino'] = Chicago_Neigh['Not Hispanic or Latino but White alone']+Chicago_Neigh['Not Hispanic or Latino but Black or African American alone']+Chicago_Neigh['Not Hispanic or Latino but American Indian and Alaska Native alone']+Chicago_Neigh['Not Hispanic or Latino but Asian alone']+Chicago_Neigh['Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone']+Chicago_Neigh['Not Hispanic or Latino but Some Other Race alone']+Chicago_Neigh['Not Hispanic or Latino but Two or More Races']
Chicago_Neigh.drop(['GeogKey','Not Hispanic or Latino but White alone','Not Hispanic or Latino but Black or African American alone','Not Hispanic or Latino but American Indian and Alaska Native alone','Not Hispanic or Latino but Asian alone','Not Hispanic or Latino but Native Hawaiian and Other Pacific Islander alone','Not Hispanic or Latino but Some Other Race alone','Not Hispanic or Latino but Two or More Races'],axis = 1, inplace = True)
Chicago_Neigh.rename(columns={'Geog':'COMMUNITY NAME'}, inplace=True)
Chicago_Neigh['COMMUNITY NAME'] =  Chicago_Neigh['COMMUNITY NAME'].str.upper()
Chicago_Neigh['Hispanic or Lation % of Total'] = Chicago_Neigh['Hispanic or Latino']/Chicago_Neigh['Total Population'] * 100
Chicago_Neigh.shape

(77, 9)

In [5]:
# Cleaning up of the datasets - Second Demographics data
Chicago_Neigh_Demo.rename(columns={'COMMUNITY AREA NAME':'COMMUNITY NAME'}, inplace=True)
Chicago_Neigh_Demo.drop(['Community Area Number','PERCENT OF HOUSING CROWDED','PERCENT HOUSEHOLDS BELOW POVERTY','PERCENT AGED 16+ UNEMPLOYED','PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA','HARDSHIP INDEX'],axis=1, inplace=True)
Chicago_Neigh_Demo['COMMUNITY NAME'] =  Chicago_Neigh_Demo['COMMUNITY NAME'].str.upper()
Chicago_Neigh_Demo.set_index("COMMUNITY NAME")
Chicago_Neigh_Demo.reset_index()
print(Chicago_Neigh_Demo.dtypes)
Chicago_Neigh_Demo.head()

COMMUNITY NAME                       object
PERCENT AGED UNDER 18 OR OVER 64    float64
PER CAPITA INCOME                     int64
dtype: object


,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME
0,ALBANY PARK,32.0,21323
1,ARCHER HEIGHTS,39.2,16134
2,ARMOUR SQUARE,38.3,16148
3,ASHBURN,36.9,23482
4,AUBURN GRESHAM,41.9,15528


In [6]:
# After careful research online, I couldnt find a geocodes dataset for the communities, however I found a shapefile with an option to export to a CSV. Downloaded the CSV and
# altered the file to bring it to a format I need.
#Import GeoCodes CSV
Chicago_geocode = pd.read_csv('Chicago_Lat_Long.csv')
Chicago_geocode.rename(columns={'Community Name':'COMMUNITY NAME'}, inplace=True)
Chicago_geocode['COMMUNITY NAME'] =  Chicago_geocode['COMMUNITY NAME'].str.upper()


## In this case we are merging both result sets to make it ready for 
Chicago_Neigh_stg = pd.merge(Chicago_Neigh_Demo, Chicago_Neigh, on='COMMUNITY NAME', how='outer')
Chicago_Neigh_Full = pd.merge(Chicago_Neigh_stg, Chicago_geocode, on='COMMUNITY NAME', how='outer')
Chicago_Neigh_Full.set_index('COMMUNITY NAME')
Chicago_Neigh_Full.head(5)

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969


### Step 2: Create three Ranking columns

    1) Rank on the Hispanic and latino population counts 

    2) Rank on Income of the community 

    3) Rank on the age groups

In [7]:
Chicago_Neigh_Full['Income_Rank'] = Chicago_Neigh_Full['PER CAPITA INCOME '].rank(ascending = 0) 
Chicago_Neigh_Full['H&L_Pop%_Rank'] = Chicago_Neigh_Full['Hispanic or Lation % of Total'].rank(ascending = 0) 
Chicago_Neigh_Full['Age_Rank'] = Chicago_Neigh_Full['PERCENT AGED UNDER 18 OR OVER 64'].rank(ascending = 0) 

####Creating another column which sums the Income rank and Hispanic and Lation Population % ranks as well

Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'] = Chicago_Neigh_Full['Income_Rank']+Chicago_Neigh_Full['H&L_Pop%_Rank']
Chicago_Neigh_Full.head(5)

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929,40.0,18.0,59.0,58.0
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579,57.0,10.0,31.0,67.0
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782,56.0,58.0,38.0,114.0
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637,34.0,24.0,45.0,58.0
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969,60.0,75.0,14.0,135.0


In [8]:
Chicago_Neigh_Full = Chicago_Neigh_Full.fillna(0)
Chicago_Neigh_Full['Income_Rank'] = Chicago_Neigh_Full['Income_Rank'].astype(int)
Chicago_Neigh_Full['H&L_Pop%_Rank'] = Chicago_Neigh_Full['H&L_Pop%_Rank'].astype(int)
Chicago_Neigh_Full['Age_Rank'] = Chicago_Neigh_Full['Age_Rank'].astype(int)
Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'] = Chicago_Neigh_Full['Income_&_Hispanic_Pop_Combined_Rank'].astype(int)
Chicago_Neigh_Full.dtypes
# Chicago_Neigh_Full.sort_values(['Income_Rank','H&L_Pop%_Rank','Age_Rank'],ascending=True)

COMMUNITY NAME                          object
PERCENT AGED UNDER 18 OR OVER 64       float64
PER CAPITA INCOME                        int64
Total Population                       float64
Hispanic or Latino                     float64
Median Age                             float64
Total Households                       float64
Male: 15 to 70 yrs                     float64
Female: 15 to 70 yrs                   float64
Not Hispanic or Latino                 float64
Hispanic or Lation % of Total          float64
Latitude                               float64
Longitude                              float64
Income_Rank                              int64
H&L_Pop%_Rank                            int64
Age_Rank                                 int64
Income_&_Hispanic_Pop_Combined_Rank      int64
dtype: object

In [9]:
####Performing some clean ups.
### In the demographics file there is a a row with "Community Name" == Chicago, it is not needed so taking it off
indexNames = Chicago_Neigh_Full[(Chicago_Neigh_Full["COMMUNITY NAME"] == "CHICAGO")].index
Chicago_Neigh_Full.drop(indexNames , inplace=True)
Chicago_Neigh_Full.head(5)

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929,40,18,59,58
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579,57,10,31,67
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782,56,58,38,114
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637,34,24,45,58
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969,60,75,14,135


In [10]:
## Adding Lat and Long information of all the communities
neighbourhood_latitude = Chicago_Neigh_Full.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = Chicago_Neigh_Full.loc[0, 'Longitude'] # neighborhood longitude value
neighbourhood_name = Chicago_Neigh_Full.loc[0, 'COMMUNITY NAME'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of ALBANY PARK are 41.961451000000004, -87.696929.


### Step3 : Using Foursquare API to get the Neighbourhood Mexican restuarants and their price tag

In [11]:
CLIENT_ID = '0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5' # your Foursquare ID
CLIENT_SECRET = 'VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5
CLIENT_SECRET:VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS


In [12]:
LIMIT = 5
radius = 500
categoryId='4bf58dd8d48988d1c1941735'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&categoryId={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighbourhood_latitude, neighbourhood_longitude, VERSION, categoryId, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=0MWLXE21XWIGAFGOXET21G53OQXRSGUSCV5DFZWDTA5ESPJ5&client_secret=VPTKIT4VQCH5VDZS0OXN2MJIX2JE0LP1ZRADXFQTUUXE55ZS&ll=41.961451000000004,-87.696929&v=20180605&categoryId=4bf58dd8d48988d1c1941735&radius=500&limit=5


In [13]:
!pip install geocoder
!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # import geocoder
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

     |████████████████████████████████| 102kB 3.2MB/s ta 0:00:011
     |████████████████████████████████| 829kB 20.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 14.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
     |████████████████████████████████| 102kB 2.6MB/s ta 0:00:011
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


In [14]:
import folium # map rendering library
latitude = 41.961451
longitude = -87.6298
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, community in zip(Chicago_Neigh_Full['Latitude'], Chicago_Neigh_Full['Longitude'], Chicago_Neigh_Full['COMMUNITY NAME']):
    label = '{}'.format(community)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  

print('##### Here is a representation of Communities on a map####')
map_chicago

##### Here is a representation of Communities on a map####


In [15]:
###Get the mexican restuarants 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId=4bf58dd8d48988d1c1941735&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            v['venue']['id'],
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['id',
                  'COMMUNITY NAME', 
                  'COMMUNITY Latitude', 
                  'COMMUNITY Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
Community_Mex_Rest1 = getNearbyVenues(names="ALBANY PARK",
                                   latitudes='41.961451',
                                   longitudes='-87.696929'
                                  )
Community_Mex_Rest1

A


KeyError: 'groups'

### List of Mexican restuarants

In [17]:
Community_Mex_Rest = getNearbyVenues(names=Chicago_Neigh_Full['COMMUNITY NAME'],
                                   latitudes=Chicago_Neigh_Full['Latitude'],
                                   longitudes=Chicago_Neigh_Full['Longitude']
                                  )
Community_Mex_Rest

ALBANY PARK
ARCHER HEIGHTS
ARMOUR SQUARE
ASHBURN
AUBURN GRESHAM
AUSTIN
AVALON PARK
AVONDALE
BELMONT CRAGIN
BEVERLY
BRIDGEPORT
BRIGHTON PARK
BURNSIDE
CALUMET HEIGHTS
CHATHAM
CHICAGO LAWN
CLEARING
DOUGLAS
DUNNING
EAST GARFIELD PARK
EAST SIDE
EDGEWATER
EDISON PARK
ENGLEWOOD
FOREST GLEN
FULLER PARK
GAGE PARK
GARFIELD RIDGE
GRAND BOULEVARD
GREATER GRAND CROSSING
HEGEWISCH
HERMOSA
HUMBOLDT PARK
HYDE PARK
IRVING PARK
JEFFERSON PARK
KENWOOD
LAKE VIEW
LINCOLN PARK
LINCOLN SQUARE
LOGAN SQUARE
THE LOOP
LOWER WEST SIDE
MCKINLEY PARK
MONTCLARE
MORGAN PARK
MOUNT GREENWOOD
NEAR NORTH SIDE
NEAR SOUTH SIDE
NEAR WEST SIDE
NEW CITY
NORTH CENTER
NORTH LAWNDALE
NORTH PARK
NORWOOD PARK
OAKLAND
O'HARE
PORTAGE PARK
PULLMAN
RIVERDALE
ROGERS PARK
ROSELAND
SOUTH CHICAGO
SOUTH DEERING
SOUTH LAWNDALE
SOUTH SHORE
UPTOWN
WASHINGTON HEIGHTS
WASHINGTON PARK
WEST ELSDON
WEST ENGLEWOOD
WEST GARFIELD PARK
WEST LAWN
WEST PULLMAN
WEST RIDGE
WEST TOWN
WOODLAWN


,id,COMMUNITY NAME,COMMUNITY Latitude,COMMUNITY Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,517c6fe4e4b019c93a3978af,ALBANY PARK,41.961451,-87.696929,Emma's Fruits @ Horner Park,41.959496,-87.698189,Mexican Restaurant
1,4bdc8a3a2a3a0f47cd50b3b6,ALBANY PARK,41.961451,-87.696929,Que Rico,41.959259,-87.699812,Mexican Restaurant
2,4b48d32df964a520c55826e3,ARCHER HEIGHTS,41.807749,-87.722579,Birrieria Zaragoza,41.804456,-87.723563,Mexican Restaurant
3,4e98ac36775b6787c26bcf80,ARCHER HEIGHTS,41.807749,-87.722579,El Asador,41.807675,-87.726896,Mexican Restaurant
4,548dff00498eecad7325ffb8,ARCHER HEIGHTS,41.807749,-87.722579,Machetes Big Quesadillas,41.804455,-87.720035,Mexican Restaurant
...,...,...,...,...,...,...,...,...
136,4e879cdc93adfd051d6d609e,WOODLAWN,41.880820,-87.620640,Wildberry Pancakes & Cafe,41.884412,-87.623047,Breakfast Spot
137,55b112d0498e1605afe59bd4,WOODLAWN,41.880820,-87.620640,LaBamba,41.881561,-87.626136,Burrito Place
138,49d8159cf964a520a05d1fe3,WOODLAWN,41.880820,-87.620640,Miller's Pub,41.879806,-87.625966,Pub
139,553977b7498eb9779dcb0592,WOODLAWN,41.880820,-87.620640,Goddess and The Baker,41.881305,-87.626117,Café


### Above step concludes that there are 141 restuarants in Chicago altogether that are tagged "Mexican"

In [18]:
Community_Mex_Rest.groupby('COMMUNITY NAME').count()
Chicago_Neigh_Full['Count of Mexican Restuarants'] = Community_Mex_Rest.groupby(['COMMUNITY NAME'])['id'].transform('count')
Chicago_Neigh_Full

                    id  COMMUNITY Latitude  COMMUNITY Longitude  Venue  \
COMMUNITY NAME                                                           
ALBANY PARK          2                   2                    2      2   
ARCHER HEIGHTS       5                   5                    5      5   
ARMOUR SQUARE        2                   2                    2      2   
ASHBURN              1                   1                    1      1   
AUSTIN               1                   1                    1      1   
AVALON PARK          2                   2                    2      2   
AVONDALE             4                   4                    4      4   
BRIDGEPORT           3                   3                    3      3   
BRIGHTON PARK        4                   4                    4      4   
CHATHAM              1                   1                    1      1   
CHICAGO LAWN         1                   1                    1      1   
DUNNING              3                

,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank,Count of Mexican Restuarants
0,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929,40,18,59,58,2
1,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579,57,10,31,67,2
2,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782,56,58,38,114,5
3,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637,34,24,45,58,5
4,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969,60,75,14,135,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,WEST LAWN,39.6,16907,33355.0,26669.0,30.000717,8766.0,11550.0,11203.0,6686.0,79.955029,41.777732,-87.713878,52,6,25,58,5
74,WEST PULLMAN,42.1,16563,29651.0,1509.0,33.928071,9492.0,9190.0,11370.0,28142.0,5.089204,41.672371,-87.649842,54,52,13,106,1
75,WEST RIDGE,38.5,23040,71942.0,14701.0,35.062344,24755.0,25321.0,25232.0,57241.0,20.434517,41.992071,-87.675487,36,31,37,67,5
76,WEST TOWN,21.7,43198,82236.0,23901.0,31.257042,38190.0,34955.0,32724.0,58335.0,29.063914,41.899386,-87.696928,9,26,75,35,5


In [54]:
##Cluster the communities

# set number of clusters
kclusters = 7

Chicago_Mex_clustering = Chicago_Neigh_Full.drop('COMMUNITY NAME', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Chicago_Mex_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
#print(Chicago_Mex_clustering.shape)
#print(Chicago_Neigh_Full.shape)

array([4, 0, 0, 5, 5, 3, 0, 4, 6, 1], dtype=int32)

In [55]:
Chicago_Neigh_Full.drop(['Cluster Labels'],axis=1,inplace=True)
Chicago_Neigh_Full.insert(0, 'Cluster Labels', kmeans.labels_)

In [56]:
latitude = 41.961451
longitude = -87.6298
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Chicago_Neigh_Full['Latitude'], Chicago_Neigh_Full['Longitude'], Chicago_Neigh_Full['COMMUNITY NAME'], Chicago_Neigh_Full['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [57]:
Chicago_Neigh_Full.head(5)

,Cluster Labels,COMMUNITY NAME,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank,Count of Mexican Restuarants
0,4,ALBANY PARK,32.0,21323,51542.0,25487.0,31.078026,16322.0,19715.0,18065.0,26055.0,49.448993,41.961451,-87.696929,40,18,59,58,2
1,0,ARCHER HEIGHTS,39.2,16134,13393.0,10182.0,30.507134,3844.0,4734.0,4483.0,3211.0,76.024789,41.807749,-87.722579,57,10,31,67,2
2,0,ARMOUR SQUARE,38.3,16148,13391.0,464.0,41.118083,5247.0,4725.0,4902.0,12927.0,3.465014,41.853035,-87.630782,56,58,38,114,5
3,5,ASHBURN,36.9,23482,41081.0,15132.0,33.483019,12451.0,13771.0,14939.0,25949.0,36.834546,41.749095,-87.679637,34,24,45,58,5
4,5,AUBURN GRESHAM,41.9,15528,48743.0,459.0,36.969525,17603.0,14402.0,18888.0,48284.0,0.941674,41.752451,-87.652969,60,75,14,135,5


In [58]:
Chicago_Neigh_Full.groupby('Cluster Labels').mean()

,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,Total Population,Hispanic or Latino,Median Age,Total Households,Male: 15 to 70 yrs,Female: 15 to 70 yrs,Not Hispanic or Latino,Hispanic or Lation % of Total,Latitude,Longitude,Income_Rank,H&L_Pop%_Rank,Age_Rank,Income_&_Hispanic_Pop_Combined_Rank,Count of Mexican Restuarants
Cluster Labels,,,,,,,,,,,,,,,,,
0,39.982609,18255.478261,13842.652174,4786.347826,34.308775,4885.521739,4618.173913,5031.826087,9056.304348,27.348318,41.805480,-87.657958,50.304348,40.739130,26.173913,91.043478,3.478261
1,30.688889,46191.111111,21653.777778,1667.222222,36.280586,10011.444444,8101.000000,8672.333333,19986.555556,7.391986,41.855365,-87.674115,9.444444,47.222222,53.000000,56.666667,3.111111
2,20.366667,73426.000000,79656.000000,4914.333333,31.499598,46558.333333,32767.666667,35204.666667,74741.666667,6.044935,41.919770,-87.645283,2.333333,49.666667,74.666667,52.000000,4.000000
3,28.970000,29563.700000,66437.200000,14618.100000,33.850903,28134.900000,25417.500000,25664.800000,51819.100000,21.439645,41.922839,-87.677265,26.800000,34.500000,60.200000,61.300000,4.200000
4,36.240000,16699.900000,45487.700000,28616.900000,29.135491,13804.800000,16296.400000,15757.200000,16870.800000,65.220860,41.857563,-87.688727,55.000000,11.300000,41.900000,66.300000,3.200000
5,38.620000,21199.550000,33304.250000,3956.000000,35.952521,12214.800000,10780.200000,12515.400000,29348.250000,11.256852,41.804694,-87.672414,42.750000,51.100000,29.700000,93.850000,3.150000
6,35.550000,12931.500000,79015.500000,63779.000000,28.931693,19861.000000,31174.500000,24929.500000,15236.500000,80.710711,41.885251,-87.740956,69.000000,6.000000,49.500000,75.000000,3.500000


# Final Results Or what I learnt from the model clusters:

Cluster 3 has the best balance between Income and Hispanic/Latino population which is what my friend is looking for.

Among the cluster 3 communities, two communities appear to best fit the need:

1) West Town

2) Logan Square

### Summary of reasons:

1) There is good Hispanic and Latino Population

2) There is good income levels that can help start a moderate price range restuarant

3) There is good age range that eats outside